# Mesoglea characterization pipeline

*written by B. Gideon Bergheim (Özbek Group, Center for Organismal Studies, Uni Heidelberg)*

This pipeline can be used to *in silico* identify mesoglea(extracellular matrix) proteins from proteome data. 

## Requirements

**Biopython**

Used for sequence manipulation.

`pip install biopython`

In [5]:
#load packages
from Bio import SeqIO

## Input

In most cases the input should be the proteome of a cnidarian species in a single .fasta file. Ideally the name of the fasta file should be the species name e.g. Hydra_vulgaris.fasta 

In [7]:
#Read the input file
SeqIO.parse("input\Hydra_vulgaris.fasta", "fasta")